This file is for our proposal notebook

# Bank Marketing Data Set Group Project Proposal
##### By: Alex Khadra, Shay Tannne, Liam Buchan, Tanaya Parakh

### Introduction:
The data set we are working on is the bank marketing data set. The bank we are analyzing is a Portuguese banking institution with data related with direct marketing campaigns (phone calls). 

The classification goal for this project, is to predict if the client will subscribe a term deposit. A term deposit is a cash investment held at a financial institution. Your money is invested for an agreed rate of interest over a fixed amount of time, or term. Term deposits can be invested into a bank, building society or credit union (in our case the Portuguese banking institution).

Our data set includes: 
- age 
- job 
- marital
- education 
- default 
- balance 
- housing 
- loan 
- contact 
- day 
- month 
- duration 
- campaign 
- pdays 
- previous 
- poutcome 
- "y" (where "y" is whether or not they got a term deposit)

Throughout the project we will look at all the variables and see how they relate to clients subscribing to a term deposit, and figure out how to best predict whether or not a client will subscribe a term deposit.